In [2]:
import numpy as np
import pandas as pd
import csv
import copy
from operator import sub
from operator import add
global bestKnow
bestKnow=[]

global ab
ab = []

test=[]

ab = pd.read_csv('../data/dC_01.csv', header=None).values[0]
bestKnow = pd.read_csv('../data/Best_Know1Fxs.csv', header=None).values[0]


npop = 20
global t, num_fun
global sumcv_times, best_times
lowerLimit = []
upperLimit = []
List_best_times=[]
List_best_gens=[]
List_sumcv_gens=[]
List_sumcv_times=[]
List_off_errors = []
x=[]
t=0
best_times=[]
sumcv_times=[] #CR was 0.2 changed to 0.4
TIMEs = 100; CR = 0.3; betamin = .2; betamax = .8; D=10; MAX_Functions = 1; runs = 1 ;frequency = 100000
middlepop = npop / 2; numChange = 100;  max_t = numChange - 1
max_eval = TIMEs * frequency + 1000; numVec = 3; num_fun=1
##lowerlimit and upperlimit set
for i in range(D):
    lowerLimit.append(-5)
    upperLimit.append(5)

def get_fitness(indv_in, num_fun, t):
    x = indv_in
    epsilon = 0.0001
    fx = 0
    sum_r = 0
    g = []
    h = []
#making objective function (fx)
    for i in range(D):
        fx = fx + x[i] ** 2
#making inequality constraint
    ga = 0
    for i in range(0, D):
        ga = ga + (1/np.sqrt(D)) * x[i]
    #print(t)
    ga = -1 * ab[t]+ ga
    g.append(ga)
    g=np.array(g)
   #calculating Sum of constraints
    for i in range(len(g)):
        if g[i] > 0:
            sum_r = sum_r + g[i]
    for i in range(len(h)):
        if abs(h[i] - epsilon) > 0:
            sum_r = sum_r + abs(h[i])
    #output of get_fitness like a structure
    indv = {
        'x':x,
        'g': g,
        'h': h,
        'fx': fx,
        'sum_r': sum_r
    }
    return indv


def make_pop(D):
    population = []
    for i in range(npop):
        x = []
        for j in range(0, D):
            x.append(np.random.uniform(lowerLimit[j], upperLimit[j]))
        individual= get_fitness(x, num_fun, t)
        population.append(individual)
    return population

def get_dif_const(g_old, g_new):
    #befor was len(g_old)
    for i in range(len(g_new)):
        if g_old[i] != g_new[i]:
            return True
    return False

def detect_change(ind):
    g_new=[]
    #g_old=[]
    g_old=ind['g']
    fx_old = ind['fx']
    #print(ind['g'])
    #for k in range(0, len(ind['g'])):
        #initialize like this for lists not like assignment
       # g_old.append(ind['g'][k])

    ind2=get_fitness(ind['x'], num_fun, t)
    for k in range(len(ind2['g'])):
        g_new.append(ind2['g'][k])
        if ind2['fx'] != fx_old or get_dif_const(g_old, g_new):
            return True
    return False

def get_best(vecU, ind2):
    if (vecU['sum_r'] == 0 and ind2['sum_r'] == 0): #if both are feasible

        if vecU['fx'] <= ind2['fx']:
            return vecU
        else:
            return ind2
        # if one of them is feasible
    if vecU['sum_r'] == 0 and ind2['sum_r']> 0:
        return vecU
    if vecU['sum_r'] > 0 and ind2['sum_r'] == 0:
        return ind2
    # both of them are infeasible
    if vecU['sum_r'] <= ind2['sum_r']:
        return vecU
    return ind2

def copy_individual(ind, i, population):
    for j in range(D):
        population[i]['x'][j] = ind['x'][j]

    population[i]['fx'] = ind['fx']
    population[i]['sum_r'] = ind['sum_r']
    for m in range(len(ind['g'])):
        population[i]['g'][m]=ind['g'][m]

def change_time(ev, freq, max_eval, max_t, bestone):
    global t
    if (ev % freq == 0) and (ev < max_eval) and (ev > 1000) and (t < max_t):
        best_times.append(bestone['fx'])
        sumcv_times.append(bestone['sum_r'])
        t = t + 1
        print(t)


def get_worst (vecU, ind2):
    # both Individuals are feasible
    if vecU['sum_r'] == 0 and ind2['sum_r'] == 0:
        if vecU['fx'] > ind2['fx']:
            return vecU
        else:
            return ind2
    # If one of them is feasible

    if vecU['sum_r'] == 0 and ind2['sum_r'] > 0:
        return ind2
    if vecU['sum_r'] > 0 and ind2['sum_r'] == 0:
        return vecU

    # both of them are infeasible
    if vecU['sum_r'] > ind2['sum_r']:
        return vecU
    return ind2

def get_worst_pop(population):
    worst = copy.deepcopy(population[0])
    for i in range(1,npop):
        #worst = copy.deepcopy(get_worst(copy.deepcopy(worst), population[i])) based on yaneli's code in java!twice clone!remove one of them
        worst = get_worst(worst, population[i])
    return worst

def getModifiedError(best, bestKnow, population):
    if best['sum_r'] > 0:  # If the best solution isn't feasible is replaced by the worst
        best = get_worst_pop(population)

    return abs(best['fx'] - bestKnow)

def get_average(mod_off_error):
    average = 0
    list_average=[]
    for i in range(len(mod_off_error)):
        average=average+mod_off_error[i]
    if average>0:
        average= average/(len(mod_off_error))
    list_average.append(average)
    return list_average

def vector_index(index):
    indp = list(range(npop))
    indp.pop(index)
    vectors=np.random.choice(indp,numVec,replace=False)
    return vectors

def main():

    startFunc = 1
    # for num_fun=startFunc in range(MAX_Functions):
    for run in range(runs):

        best_gens = [] #save best fitness of every generation
        sumcv_gens = []#sum of constraint violation
        mod_off_error = []

        gen = 0
        ev = 0
        global t
        t = 0

        #a random population is created
        # caluculate getfitness for all population and save in structure named population
        population = make_pop(D)
        #print(population[1])

        #get the best pop 
        bestone = population[0]

        for i in range(1, npop):
            bestone = get_best(bestone, population[i])

        while ev < max_eval and t < TIMEs:
            #print(population[0])
            gen=gen+1
            #measurement:modified offline error
            #We get the modified error per generation
            mod_off_error.append(getModifiedError(bestone, bestKnow[t], population))
            #general cycle of evolution
            for i in range(npop):
                #print(i)
                #print(population[i])
   ###simple detect change mechanism for first and midlle individual in population
                if i == 0 or i == middlepop:
                    if detect_change(population[i]):
                        #simple re-intialization

                        for j in range(npop):
                            population[j] = get_fitness(population[j]['x'], num_fun, t)
                            ev = ev+1
                            change_time(ev, frequency, max_eval, max_t, bestone)
                        bestone = get_fitness(bestone['x'], num_fun, t)
                        ev = ev+1
                        change_time(ev, frequency, max_eval, max_t, bestone)
                        ###??????? i dont know if we have to include this or not check more carefuly
                 #   ev = ev + 1
                  #  change_time(ev, frequency, max_eval, max_t, bestone)

    ### end of change detection mechanism

    ###start to evolve the population

    #chose three differents vectors:three individuals other than i are chosen
                vectors = vector_index(i)
                jrand = np.random.randint(D)
                ind = {'x':[], 'g':[], 'h':[], 'fx':[]}
               
                for j in range(D):
                    F = np.random.uniform(betamin, betamax)
                    if np.random.rand() < CR or j == jrand:
                        ind['x'].append(population[vectors[0]]['x'][j]+
                                        F * (population[vectors[1]]['x'][j]
                                             - population[vectors[2]]['x'][j]))                      
        ##make sure the created x is in range of the variable
                        ind['x'][j] = max(ind['x'][j], lowerLimit[j])
                        ind['x'][j] = min(ind['x'][j], upperLimit[j])
                    else:
                        ind['x'].append(population[i]['x'][j])

        ###x of the new individual has been created, go for calculate its details(obj fun and sumCv)
                ind = get_fitness(ind['x'], num_fun, t) 
                ev = ev+1
                change_time(ev, frequency, max_eval, max_t, bestone)
                ind = copy.deepcopy(get_best(ind, population[i]))
        ### Replace the information of the ith population with the new individual
                copy_individual(ind, i, population)
        ###Verify if the new individual is the best
                bestone = copy.deepcopy(get_best(bestone, population[i]))

                if ev >= max_eval:
                    break
            ###end evolve population
            ###save data of this generation
            best_gens.append(bestone['fx'])
            sumcv_gens.append(bestone['sum_r'])
    #end of the runs
        List_best_times.extend(best_times)
        List_best_gens.extend(best_gens)
        List_sumcv_gens.extend(sumcv_gens)
        List_sumcv_times.extend(sumcv_times)
        List_off_errors.extend(get_average(mod_off_error))
    print(List_best_times)
    print(len(List_best_times))
    print(List_sumcv_times)


main()
def write_csv(name):
    with open(name, mode='w') as const_file:
        const_writer = csv.writer(const_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        const_writer.writerow(List_best_times)

write_csv('List_best_times.csv')


1
2
3


KeyboardInterrupt: 